# Belief Propagation from Geo-Located Imagery

In [1]:
# If new server on Descartes Labs, need to install rioxarray 
try: import rioxarray
except: 
    %pip install rioxarray
    
# Import helper functions
import imports as ip
import netconf as nc
import plotting as pl
import helper_functions as hf

In [2]:
def parameter_input():
    import ipywidgets as ipw
    ## Define variables
    # Label imports
    layout = {'width': 'max-content'}
    display(ipw.HTML(value = f"<b>{'Label Parameters'}</b>"))
    bxLabel = ipw.Box([ipw.Label(value='Damage Labels: Shapefile - '), ipw.Text(value='./data/beirutDamages.shp', placeholder='damages.shp',  disabled=False,layout=layout),
                   ipw.Label(value='Coordinate System - '), ipw.Text(value='EPSG:4326', placeholder='EPSG:4326',  disabled=False,layout=layout),
                   ipw.Label(value='Decision Column - '), ipw.Text(value='decision', placeholder='decision',  disabled=False, layout=layout),
                   ipw.Label(value='First Word Only - '), ipw.Checkbox(value=False, disabled=False, indent=False)])

    bxConf = ipw.Box([ipw.Label(value='Label Confidence ($P_{other label}$, $P_{class}$)'),
                      ipw.FloatRangeSlider(value=[0, 1], min=0, max=1, step=0.01, disabled=False, continuous_update=True, orientation='horizontal', readout=True, readout_format='.2f')])
    display(bxLabel,bxConf)

    # Data imports
    display(ipw.HTML(value = f"<b>{'Data Parameters'}</b>"))
    bxDataTypes = ipw.Box([ipw.Label(value='Enter Data Types:'),
                           ipw.Combobox(value='HighRes Imagery', placeholder='Data Type 1', options=['HighRes Imagery','Interferogram','LowRes Imagery'], ensure_option=False, disabled=False,layout=layout),
                           ipw.Combobox(value='Interferogram', placeholder='Data Type 2', options=['HighRes Imagery','Interferogram','LowRes Imagery'], ensure_option=False, disabled=False,layout=layout),
                           ipw.Combobox(placeholder='Data Type 3', options=['HighRes Imagery','Interferogram','LowRes Imagery'], ensure_option=False, disabled=False,layout=layout),
                           ipw.Combobox(placeholder='Data Type 4', options=['HighRes Imagery','Interferogram','LowRes Imagery'], ensure_option=False, disabled=False,layout=layout)])

    button1 = ipw.Button(description='Confirm Types', disabled=False, button_style='success', tooltip='Confirm', icon='check')

    bxMap = ipw.Box([ipw.Label(value='Latitude - '), ipw.FloatText(value=33.893, placeholder='dd.dddd',  disabled=False,layout=layout),
                       ipw.Label(value='Longitude - '), ipw.FloatText(value=35.512, placeholder='dd.dddd',  disabled=False,layout=layout),
                       ipw.Label(value='Zoom - '), ipw.IntText(value=14, placeholder='dd.dddd',  disabled=False, layout=layout),
                       ipw.Label(value='Standard Test Area - '), ipw.Checkbox(value=False, disabled=False, indent=False)])

    display(bxDataTypes)

    defFiles = ["data/highRes/20JUL31_HR_LatLon.tif","data/highRes/20AUG05_HR_LatLon.tif","./data/beirutPrePreExplosionIfg.tif","./data/beirutPrePostExplosionIfg.tif"]

    out1 = ipw.Output()
    def on_button1_clicked(b1):
        button1.description = 'Confirmed'
        with out1:
            dataTypes = [i.value.split(' ')[0] for i in bxDataTypes.trait_values()['children'][1:] if len(i.value) > 0]
            for i in range(len(dataTypes)):
                try: globals()['bxfile'+str(i)] = ipw.Box([ipw.Label(value=dataTypes[i]+' File Locations: Pre -'), ipw.Text(value=defFiles[2*i], placeholder=dataTypes[i]+'PreFile', disabled=False),
                         ipw.Label(value=' Post -'), ipw.Text(value=defFiles[2*i+1], placeholder=dataTypes[i]+'PostFile', disabled=False)])
                except: globals()['bxfile'+str(i)] = ipw.Box([ipw.Label(value=dataTypes[i]+' File Locations: Pre -'), ipw.Text(placeholder='Enter file path', disabled=False),
                         ipw.Label(value=' Post -'), ipw.Text(placeholder='Enter file path', disabled=False)])
                display(globals()['bxfile'+str(i)])
                v['bxfile'+str(i)] = globals()['bxfile'+str(i)]
            display(ipw.HTML(value = f"<b>{'Map Properties'}</b>"))
            display(bxMap)
            
    button1.on_click(on_button1_clicked)
    display(ipw.VBox([button1, out1]))
    
    v = {}
    v.update({'bxLabel':bxLabel,'bxConf':bxConf,'bxDataTypes':bxDataTypes,'bxMap':bxMap})

    return v

In [3]:
import ipywidgets as ipw
def label_map(v):
    # Extract parameter values from widgets
    v['groundTruth'], v['crs'], v['cn'], v['splitString'] =  [i.value for i in v['bxLabel'].trait_values()['children'][1::2]]
    v['confidence'] = list(v['bxConf'].trait_values()['children'][1].value)
    v['dataTypes'] = [i.value.split(' ')[0] for i in v['bxDataTypes'].trait_values()['children'][1:] if len(i.value) > 0]
    for j in range(len(v['dataTypes'])):
        v['preFile'+str(j)], v['postFile'+str(j)] = [i.value for i in v['bxfile'+str(j)].trait_values()['children'][1::2]]
    v['lat'], v['lon'], v['zoom'], v['stdTest'] = [i.value for i in v['bxMap'].trait_values()['children'][1::2]]

    ## Import Labels and combine
    labels = ip.shape_to_gdf(v['groundTruth'], v['splitString'], v['cn'], crs=v['crs'])

    # Display map of assessments upon which to draw Polygon for analysis
    m1 = pl.create_map(v['lat'], v['lon'], v['zoom'])
    m1 = pl.plot_assessments(labels, m1)
    m1, testPoly = pl.draw_polygon(labels, m1, v['stdTest'])
    display(m1)
    v['labels'], v['testPoly'] = labels, testPoly
    return v

def model_parameters(v):
    layout = {'width': 'max-content'}
    display(ipw.HTML(value = f"<h3>{'Model Parameters'}</h3>"))
    display(ipw.HTML(value = f"<b>{'Class Properties'}</b>"))
    # Display default classes from labels
    unique = v['labels'][v['cn']].unique()
    display(ipw.HTML(value = "Label Classes - "f"{str(unique)}"))
    # Ask for number of classes to use
    bxNClasses = ipw.Box([ipw.Label(value='Classes for Model - '), ipw.Dropdown(options=list(range(2,len(unique)+1)),value=max(list(range(len(unique)+1))),disabled=False)])   
    display(bxNClasses)
    
    # Nodes
    bxNodes = ipw.Box([ipw.Label(value='Maximum nodes - Sampling occurs if < pixel number: '), ipw.IntText(value=20000, placeholder='20000',  disabled=False, step=1000, layout=layout)])

    # Edges
    # Neighbours for each data type
    bxEdges = ipw.Box([ipw.Label(value='Neighbours - Edges to nearest values for each node: '), ipw.Box([ipw.IntText(value=3, placeholder='edges', description=str(i)+' - ', disabled=False, layout=layout) for i in v['dataTypes']])])
    # Geographical neighbours
    bxAdjacent = ipw.Box([ipw.Label(value='Geographical Edges - '), ipw.Checkbox(value=False, disabled=False, indent=False, layout=layout), ipw.Label(value='Geographical Neighbours - '), ipw.IntText(value=4, placeholder='edges',  disabled=False, layout=layout)])
            
    # Once confirmed then display classification options
    button3 = ipw.Button(description='Confirm Classes', disabled=False, button_style='success', tooltip='Confirm', icon='check')
    out3 = ipw.Output()
    def on_button3_clicked(b3):
        button3.description = 'Confirmed'
        with out3:
            # Read number of classes
            nClasses = bxNClasses.trait_values()['children'][1].value
            # If class grouping required propose options
            if nClasses < len(unique):
                # Opt to use clustering or not
                bxCluster = ipw.Box([ipw.Label(value='Use class clustering - Uncheck to assign classes below:'), ipw.Checkbox(value=True, disabled=False, indent=False)])
                # Assign each value to a class
                bxAssign = ipw.Box([ipw.SelectMultiple(options=unique, rows=len(unique), description='Class '+str(i)+':', disabled=False) for i in range(nClasses)])
                # Edit class names if desired
                bxClNames = ipw.Box([ipw.Text(value=str(i), placeholder='Enter Class Name', description='Class '+str(i)+':', disabled=False) for i in range(nClasses)])
                display(bxCluster, bxAssign)
                display(ipw.HTML(value = "Edit Class Names:"))
                display(bxClNames)
                
                v.update({'bxCluster':bxCluster, 'bxAssign':bxAssign, 'bxClNames':bxClNames})
                # PCA options if needed in future
                # pca, pcaComps, meanCluster = False, 2, True # Clustering properties if used
                
            # Nodes
            display(ipw.HTML(value = f"<b>{'Node Properties'}</b>"))
            display(bxNodes)

            # Edges
            display(ipw.HTML(value = f"<b>{'Edge Properties'}</b>"))
            display(bxEdges,bxAdjacent)
                
    button3.on_click(on_button3_clicked)
    display(ipw.VBox([button3, out3]))
    v.update({'bxNClasses':bxNClasses,'bxNodes':bxNodes,'bxEdges':bxEdges,'bxAdjacent':bxAdjacent})
    
    return v   

In [4]:
# Data Imports
# Reproject data to used crs
def reproject_data(v):
    print("------Checking Coordinate Systems-------")
    for i in range(len(v['dataTypes'])):
        if v['crs'] not in ip.get_crs(v['postFile'+str(i)]):
            v['postFile'+str(i)] = ip.conv_coords([v['postFile'+str(i)]], ["data/PostConv"+str(i)+".tif"], v['crs'])[0]
            if v['preFile'+str(i)]: v['preFile'+str(i)] = ip.conv_coords(v['preFile'+str(i)], ["data/PreConv"+str(i)+".tif"], v['crs'])[0]
    print("------Finished Checking Coordinate Systems-------")
    return v

def import_data(v):
    for i in v.keys(): globals()[i] = v[i]
    # Retrieve Data from inputs
    max_nodes = bxNodes.trait_values()['children'][1].value
    neighbours = [i.value for i in bxEdges.trait_values()['children'][1].trait_values()['children']]
    adjacent, geoNeighbours = [i.value for i in bxAdjacent.trait_values()['children'][1::2]]
    nClasses = bxNClasses.trait_values()['children'][1].value
    classAssign = False if 'bxAssign' not in v or bxAssign.trait_values()['children'][1].value else [list(i.value) for i in bxAssign.trait_values()['children']]
    classNames = False if 'bxClNames' not in v else [i.value for i in bxClNames.trait_values()['children']]
    v.update({'max_nodes':max_nodes, 'neighbours':neighbours, 'adjacent':adjacent, 'geoNeighbours':geoNeighbours, 'nClasses':nClasses, 'classAssign':classAssign, 'classNames':classNames})
    # Reproject Data if necessary
    v = reproject_data(v)
    
    # Import Files
    print("------Importing Data Files---------")
    # Import first data type
    df, crop = ip.img_to_df(postFile0, testPoly, crs=crs)
    if preFile0:
        preDf, _ = ip.img_to_df(preFile0, testPoly, crs=crs)
        df -= preDf

    # Import other data types
    if len(dataTypes) > 1:
        crop.rio.to_raster("croptemp.tif")
        for i in range(1, len(dataTypes)):
            ip.resample_tif(globals()['postFile'+str(i)], testPoly, 'posttemp'+str(i)+'.tif')
            globals()['dataArray'+str(i)] = ip.tif_to_array('posttemp'+str(i)+'.tif', 'resample')
            if globals()['preFile'+str(i)]: 
                ip.resample_tif(globals()['preFile'+str(i)], testPoly, 'pretemp'+str(i)+'.tif')
                globals()['dataArray'+str(i)] -= ip.tif_to_array('pretemp'+str(i)+'.tif', 'resample')
        ip.del_file_endings(".", "temp.tif")

    # Concatenate data types
    data = df.copy()
    for j in range(1, len(dataTypes)): data[dataTypes[j]]=globals()['dataArray'+str(j)].flatten()
    data.dropna(inplace=True)

    # Sample data and create geodataframe
    gdf = ip.get_sample_gdf(data, max_nodes, crs)
    v.update({'max_nodes':max_nodes, 'gdf':gdf, 'typesUsed':[list(df.columns.values), dataTypes[1:]]})
    print("------Finished Data Import---------")
    return v

In [5]:
## Assign Label classes to data
def classify_data(v):
    print("------Data Classification---------")
    for i in v.keys(): globals()[i] = v[i]
    nClasses = v['nClasses']
    defClasses, labelsUsed, dataUsed = len(labels[cn].unique()), labels.to_crs(crs).copy(), gdf.copy() # Default classes from labels
    usedNames = labels[cn].unique() if nClasses==defClasses or nClasses is False else classNames
    initial = hf.init_beliefs(dataUsed, classes=nClasses, columns=usedNames, crs=crs) # Initial class value for each data pixel

    if not nClasses or nClasses == defClasses: 
        nClasses = defClasses # If default classes used
        classesUsed = usedNames.copy()
    elif nClasses > defClasses: raise NameError('Cannot assign more classes than in original data') # If invalid input
    elif nClasses < defClasses: # Perform class grouping
        if not classAssign: # Perform clustering
            # Assign labels to each pixel
            allPixels = hf.create_nodes(initial, labelsUsed[['geometry',cn]][labelsUsed.within(hf.get_polygon(testPoly, conv=True))])
            # Run PCA if set to True
            #X = hf.run_PCA(dataUsed[typesUsed[0]].values.transpose(), pcaComps).components_.transpose() if pca else dataUsed[typesUsed[0]]
            X = dataUsed[typesUsed[0]]
            # Run clustering
            meanCluster = True
            kmeans, clusterClasses, initLabels = hf.run_cluster(X.iloc[allPixels[cn].dropna().index].values.reshape(-1,len(typesUsed[0])), allPixels[cn].dropna(), meanCluster, nClasses)
            print('Clustered classes:{} , original classes:{}'.format(clusterClasses, initLabels))
            # Create groups of classes
            classesUsed = []
            for j in range(nClasses): classesUsed.append([initLabels[i] for i, x in enumerate(list(clusterClasses)) if x==j])
        else: 
            classesUsed = classAssign
            #used = [i in flatten_list(classesUsed) for i in labelsUsed[cn]]
            initial = hf.init_beliefs(dataUsed, classes=nClasses, columns=usedNames, crs=crs)

        # Assign labels for each pixel after clustering
        labelsUsed[cn] = hf.group_classes(labelsUsed[cn], classesUsed)
    v.update({'labelsUsed':labelsUsed,'initial':initial, 'usedNames':usedNames, 'classesUsed':classesUsed})
    print("------Finished Data Classification---------")
    return v
    


In [6]:
def run_bp(v):
    for i in v.keys(): globals()[i] = v[i]
    # Split train/test set for located nodes
    X_train, X_test, y_train, y_test = hf.train_test_split(labelsUsed, cn, hf.get_polygon(testPoly, conv=True))

    # Create nodes
    nodes = hf.create_nodes(initial, X_train)

    # Assign prior beliefs from assessments
    priors = hf.prior_beliefs(nodes, beliefColumns = initial.columns[-nClasses:], classNames=classNames, column = cn)

    # Create edges
    edges = hf.create_edges(nodes, adjacent=adjacent, geo_neighbors=geoNeighbours, values=typesUsed, neighbours=neighbours)
    
    # Run belief propagation
    beliefs, _ = nc.netconf(edges,priors,verbose=True,limit=1e-3)
    
    v.update({'X_train':X_train, 'X_test':X_test, 'nodes':nodes, 'priors':priors, 'edges':edges,'beliefs':beliefs})
    return v

In [7]:
def evaluate_output(v):
    for i in v.keys(): globals()[i] = v[i]
    # Evaluation Metrics
    # Get y_true vs y_pred for test set
    y_true, y_pred = hf.get_labels(initial, X_test, beliefs, column=cn)

    # Classification metrics
    yp_clf, classes = hf.class_metrics(y_true, y_pred, classes=usedNames, orig=classNames)

    fig, axs = pl.create_subplots(1,2, figsize=[14,5])

    # Confusion matrix
    axs = pl.confusion_matrix(axs, y_true, yp_clf, classes if len(labels[cn].unique()) == nClasses else list(range(nClasses)))

    # Cross entropy / Confidence metrics
    if nClasses == 2: axs = pl.cross_entropy_metrics(axs, y_true, y_pred[:,1].reshape(-1,1), classes)
    else: axs[1] = pl.cross_entropy_multiclass(axs[1], y_true, y_pred)

    pl.show_plot()
    
    v.update({'y_true':y_true, 'y_pred':y_pred, 'yp_clf':yp_clf, 'classes':classes, 'fig':fig})
    
    return v

In [8]:
# Save figure
def save_plot(v, location=False):
    for i in v.keys(): globals()[i] = v[i]
    if location: pl.save_plot(fig, location)
    else: pl.save_plot(fig, 'results/Beirut_UN_nd{}_cls{}{}_neighbours{}{}_std{}_adj{}{}'.format(str(len(nodes)),str(nClasses),str(classesUsed),
                                                                                          str(dataTypes),str(neighbours),str(stdTest),
                                                                                          str(adjacent),str(geoNeighbours)))

__________
Let's begin with the input parameters. These include the label file, confidence in the labels and the data types we will use. Once we confirm the data types we will be asked for paths to the files containing the imagery. Post-event must be provided but pre-event is optional. If a pre-event image is provided the data used will be the difference between the images which contains more information than the post event image alone.

In [9]:
v1 = parameter_input()

HTML(value='<b>Label Parameters</b>')

Box(children=(Label(value='Damage Labels: Shapefile - '), Text(value='./data/beirutDamages.shp', layout=Layout…

Box(children=(Label(value='Label Confidence ($P_{other label}$, $P_{class}$)'), FloatRangeSlider(value=(0.0, 1…

HTML(value='<b>Data Parameters</b>')

Box(children=(Label(value='Enter Data Types:'), Combobox(value='HighRes Imagery', layout=Layout(width='max-con…

__________
Now let's load up the map of our ground labels and define an area for the model.

In [10]:
v2 = label_map(v1)

Map(center=[33.893, 35.512], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…

______
Now we'll pick the model parameter to run on the data from the selected area. If we wish to group classes together we will also be offered some clustering options.

In [11]:
v3 = model_parameters(v2)

HTML(value='<h3>Model Parameters</h3>')

HTML(value='<b>Class Properties</b>')

HTML(value="Label Classes - ['GREEN' 'YELLOW' 'LAND' 'RED' 'TOTAL']")

Box(children=(Label(value='Classes for Model - '), Dropdown(index=3, options=(2, 3, 4, 5), value=5)))

________
Now we have all the parameters for the model, let's import and classify the data according to our selections.

In [12]:
v4 = import_data(v3)
v5 = classify_data(v4)

------Checking Coordinate Systems-------
------Finished Checking Coordinate Systems-------
------Importing Data Files---------
data/highRes/20AUG05_HR_LatLon.tif read completed.
data/highRes/20JUL31_HR_LatLon.tif read completed.


/opt/conda/lib/python3.7/site-packages/IPython/core/async_helpers.py:68: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  coro.send(None)


./data/beirutPrePostExplosionIfg.tif read completed.
./data/beirutPrePreExplosionIfg.tif read completed.
------Finished Data Import---------
------Data Classification---------
------Finished Data Classification---------


____________
OK, the data is formatted the model parameters are all checked. Let's build the graph of nodes & edges and run the belief propagation!

In [13]:
v6 = run_bp(v5)

Nodes: 20000, Edges: 120000
It	Loss	Label change

0	1.55548e+00		0

1	6.06427e-02		0

2	3.68523e-01		0

3	1.41269e-02		0

4	8.72185e-02		0

5	6.10914e-03		0

6	2.08523e-02		0

7	2.48441e-03		0

8	5.03666e-03		0

9	8.37731e-04		0

Time elapsed: 7.210722208023071 seconds


_____
Now let's use the test set to evaluate the effectiveness of the model.

In [14]:
v7 = evaluate_output(v6)

ValueError: Mix of label input types (string and number)

Want to save the plot? Run the cell below. If you want to specify a location replace the False boolean with the filepath.

In [ ]:
save_plot(v7, location=False)

In [ ]:
# Visualise spatial results
fig, axs = pl.create_subplots(2,2,figsize=[15,12])

prePlot = pl.belief_plot(nodes, axs[0,0], 'RED', normalise=False)
postPlot = pl.belief_plot(nodes, axs[0,1], beliefs, normalise=True)
assessPlt = joint.loc[joint.within(poly)].plot(ax=axs[1,0], column='decision',cmap='RdYlGn_r')
ifgPlot = (pl.cropped_ifg(ifgPreFile,testPoly)-pl.cropped_ifg(ifgPostFile,testPoly)).plot(ax=axs[1,1])
prePlot.set_title('A priori damage likelihood'), postPlot.set_title('Updated damage likelihood'), assessPlt.set_title('Damage Assessments')

pl.show_plot()